In [1]:
import pandas as pd
import numpy as np
from plotnine import *

# Read in data

In [2]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

data_full = fetch_openml(
    "titanic", version=1, as_frame=True
)

data = pd.concat([data_full['data'], data_full['target']], axis = 1)

drop_cols = ['boat', 'body', 'home.dest']

data.drop(columns = drop_cols, inplace = True)

# change data types to match the csv data types in kaggle
data = data.astype({'pclass': 'int32', 'sex': 'object', 'sibsp': 'int32', 'parch': 'int32', 'fare': 'float32', 'embarked': 'object', 'survived': 'int32'})
data.head()

,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,survived
0,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.337494,B5,S,1
1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.550003,C22 C26,S,1
2,1,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.550003,C22 C26,S,0
3,1,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.550003,C22 C26,S,0
4,1,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.550003,C22 C26,S,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pclass    1309 non-null   int32  
 1   name      1309 non-null   object 
 2   sex       1309 non-null   object 
 3   age       1046 non-null   float64
 4   sibsp     1309 non-null   int32  
 5   parch     1309 non-null   int32  
 6   ticket    1309 non-null   object 
 7   fare      1308 non-null   float32
 8   cabin     295 non-null    object 
 9   embarked  1307 non-null   object 
 10  survived  1309 non-null   int32  
dtypes: float32(1), float64(1), int32(4), object(5)
memory usage: 87.1+ KB


In [4]:
numeric_cols = ['age', 'sibsp', 'parch', 'fare']
categorical_cols = ['pclass', 'sex', 'cabin', 'embarked']

In [5]:
data.describe()

,pclass,age,sibsp,parch,fare,survived
count,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000,1309.000000
mean,2.294882,29.881135,0.498854,0.385027,33.295479,0.381971
std,0.837836,14.413500,1.041658,0.865560,51.758667,0.486055
min,1.000000,0.166700,0.000000,0.000000,0.000000,0.000000
25%,2.000000,21.000000,0.000000,0.000000,7.895800,0.000000
50%,3.000000,28.000000,0.000000,0.000000,14.454200,0.000000
75%,3.000000,39.000000,1.000000,0.000000,31.275000,1.000000
max,3.000000,80.000000,8.000000,9.000000,512.329224,1.000000


# Initial data exploration

Target distribution (just get from describe)

In [6]:
data['survived'].mean()

np.float64(0.3819709702062643)

Percent null of columns (just use df.info())

In [7]:
def pct_null(df, column):
    return len(df[df[column].isna()][column]) / len(df[column])

pct_null(data, 'fare')

0.0007639419404125286

Compute correlation of features with output

In [8]:
for col in numeric_cols:
    print(f"{col}: {data[col].corr(data['survived'])}")

age: -0.05551252019214634
sibsp: -0.02782511923058188
parch: 0.08265957038609854
fare: 0.24426546684798725


In [9]:
from scipy.stats import chi2_contingency

chi2, p_val = chi2_contingency(pd.crosstab(data['pclass'], data['survived']))[:2]

print(chi2)
print(p_val)

127.85915643930326
1.7208259588256175e-28


## Check for duplicated

In [10]:
len(data['name'].unique())

1307

In [11]:
data[data['name'].duplicated()]

,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,survived
726,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,NaN,Q,0
925,3,"Kelly, Mr. James",male,44.0,0,0,363592,8.0500,NaN,S,0


In [12]:
duplicates = data[data.duplicated(subset=['name'], keep=False)]
print(duplicates)

     pclass                  name     sex   age  sibsp  parch  ticket    fare  \
725       3  Connolly, Miss. Kate  female  22.0      0      0  370373  7.7500   
726       3  Connolly, Miss. Kate  female  30.0      0      0  330972  7.6292   
924       3      Kelly, Mr. James    male  34.5      0      0  330911  7.8292   
925       3      Kelly, Mr. James    male  44.0      0      0  363592  8.0500   

    cabin embarked  survived  
725   NaN        Q         1  
726   NaN        Q         0  
924   NaN        Q         0  
925   NaN        S         0  


# Model training

In [13]:
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, KBinsDiscretizer
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer, SimpleImputer
import xgboost as xgb
from sklearn import set_config

# Can change to transform_output = 'default' to do np arrays
set_config(transform_output="pandas")

numeric_cols = ['age', 'sibsp', 'parch', 'fare']
categorical_cols = ['pclass', 'sex', 'cabin', 'embarked']
target = ['survived']

X = data[numeric_cols + categorical_cols].copy()
y = np.asarray(data[target], dtype = 'int8')

X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(X, y, test_size = 0.2, stratify = y, random_state = 20251207)

skf = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 20251210)

train_idx, val_idx = next(skf.split(X_train_full, y_train_full))

X_train = X_train_full.iloc[train_idx]
X_val = X_train_full.iloc[val_idx]
y_train = y_train_full[train_idx]
y_val = y_train_full[val_idx]


In [14]:
X.iloc[train_idx]

,age,sibsp,parch,fare,pclass,sex,cabin,embarked
0,29.0,0,0,211.337494,1,female,B5,S
2,2.0,1,2,151.550003,1,female,C22 C26,S
3,30.0,1,2,151.550003,1,male,C22 C26,S
4,25.0,1,2,151.550003,1,female,C22 C26,S
5,48.0,0,0,26.549999,1,male,E12,S
...,...,...,...,...,...,...,...,...
1041,NaN,0,0,8.050000,3,male,NaN,S
1043,NaN,1,0,15.500000,3,female,NaN,Q
1044,NaN,0,0,15.500000,3,female,NaN,Q
1045,18.0,0,0,7.750000,3,male,NaN,S


In [19]:
ct = ColumnTransformer([
   ('scaler', StandardScaler(), numeric_cols),
   ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_cols),
])

xgb_model = xgb.XGBClassifier()

pipe = Pipeline([
    ('encode_cats', ct),
    ('imputer', SimpleImputer(strategy='mean')),
    ('clf', xgb_model)
], verbose = True)

In [20]:
pipe.fit(X_train, y_train)

[Pipeline] ....... (step 1 of 3) Processing encode_cats, total=   0.1s
[Pipeline] ........... (step 2 of 3) Processing imputer, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing clf, total=   0.2s


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('encode_cats', ...), ('imputer', ...), ...]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",True
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('scaler', ...), ('encoder', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different tra

In [18]:
from sklearn.metrics import classification_report

y_pred = pipe.predict(X_val)
y_pred_proba = pipe.predict_proba(X_val)

print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.91      0.86        65
           1       0.82      0.68      0.74        40

    accuracy                           0.82       105
   macro avg       0.82      0.79      0.80       105
weighted avg       0.82      0.82      0.81       105



In [22]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_val, y_pred_proba[:,1])

0.9046153846153846